In [4]:
# KOBIS .xlsx 데이터에서 영화제목으로 네이버 영화 플랫폼에서 추가 정보 수집

In [5]:
import urllib
import pandas as pd

from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

In [6]:
# 1) kobis 데이터 획득
kobisData_raw = pd.read_excel("./data/전체_영화_데이터.xls")
kobisData = kobisData_raw[(kobisData_raw['전국스크린수']>50) & (kobisData_raw['전국관객수']>1000)]
#kobisData.head()
kobisData.set_index("영화명",inplace=True)
len(kobisData)
#kobisData.head(2)

2950

In [7]:
#2) 네이버 영화 html 코드 획득 및 확인
#html = "https://movie.naver.com/movie/search/result.naver?query={search_movie}&section=all"
html = "https://movie.naver.com/movie/search/result.naver?section=movie&query=%EB%AA%85%EB%9F%89"
req = Request(html.format(search_movie=urllib.parse.quote("힘을 내요, 미스터 리"))) # 글자를 url로 인코딩
#req = Request(html.format(search_movie=urllib.parse.quote("명량"))) # 글자를 url로 인코딩
response = urlopen(req)
response, response.status

(<http.client.HTTPResponse at 0x227184d5190>, 200)

In [8]:
soup = BeautifulSoup(response,"html.parser")
#print(soup.prettify())

In [19]:
# 2) 네이버데이터 크롤링
movieName_List = []
director_List = []
actors_List = []
score_List = []
netizen_List = []
runtime_List = []
# kobisData에서 영화이름 가져와서 
errormovie = []
for i in range(len(kobisData)):
    movieName = str(kobisData.index[i])
    # html 열고
    html = "https://movie.naver.com/movie/search/result.naver?query={search_movie}&section=all"
    req = Request(html.format(search_movie=urllib.parse.quote(movieName)))
    response = urlopen(req)
    soup = BeautifulSoup(response,"html.parser")
    #print(response, response.status)
    real_director = kobisData.iloc[i]["감독"]
    for i in range(0,len(soup.find_all(class_='etc')),2): #n개 영화면 2n까지
        try:
            tmp_director = soup.find_all(class_='etc')[i+1].get_text().split('|')[0][5:]
            if tmp_director == real_director: #감독일치하면
                movie_info_s = soup.find("ul",class_='search_list_1').find_all(class_='etc')[1].get_text().split('|')
                #print(movie_info_s)
                # 주연배우
                actors = movie_info_s[-1][5:].split(',')
                #print("actors",actors)
                # 평점
                score_info = soup.find("ul",class_='search_list_1').find('dd',class_='point')
                score = score_info.get_text()[1:5]
                # 평가자 수
                netizen = score_info.get_text().split(' ')[-1][:-3]
                # 상영시간
                runtime = soup.find("ul",class_='search_list_1').find_all(class_='etc')[0].get_text().split('|')[2][:-2]
                movieName_List.append(movieName)
                director_List.append(tmp_director)
                actors_List.append(actors)
                score_List.append(score)
                netizen_List.append(netizen)
                runtime_List.append(runtime)
                break
            else :
                if movieName not in errormovie:
                     errormovie.append(movieName)
                # print(kobisData.index[i][""])
                # print("감독명 비교 : ",tmp_director,"==",real_director)
                print(f"error count : {len(errormovie)} ",movieName)
                # print(soup.find("ul",class_='search_list_1').find_all(class_='etc')[1].get_text().split('|'))
                # print(soup.find("ul",class_='search_list_1').find_all(class_='etc')[1].get_text().split('|')[-1][5:])
                # print(score_info.get_text().split(' ')[-1][:-3])
                # print("runtime",soup.find("ul",class_='search_list_1').find_all(class_='etc')[0].get_text().split('|')[2][:-2])
                # #print(real_director,":",tmp_director,real_director==tmp_director)
                continue
        except:
            if movieName not in errormovie:
                    errormovie.append(movieName)
            print(f"error count : {len(errormovie)}",movieName)   
            continue

error count : 1  100% 울프: 푸들이 될 순 없어
error count : 2  1942: 언노운 배틀
error count : 3  28세 미성년
error count : 4  5일의 마중
error count : 5  60만번의 트라이
error count : 6  I AM
error count : 6  I AM
error count : 6  I AM
error count : 6  I AM
error count : 6  I AM
error count : 6  I AM
error count :6  I AM
error count : 7  n번째 이별 중
error count : 8  가나의 혼인잔치: 언약
error count : 9  가디언즈
error count : 9  가디언즈
error count : 9  가디언즈
error count : 10  가비
error count : 11  가시
error count : 12  가필드 펫 포스 3D
error count : 13  감기
error count : 13  감기
error count : 13  감기
error count : 14  감바의 대모험
error count : 15  감시자들
error count : 15  감시자들
error count : 15  감시자들
error count : 16  개구리왕국
error count : 17  개구리왕국 2
error count : 18  개구쟁이 스머프
error count : 19  갤로우즈
error count : 20  갬빗
error count : 21  거미줄에 걸린 소녀
error count : 22  거인
error count : 22  거인
error count : 22  거인
error count : 22  거인
error count : 23  걸후드
error count : 24  검은 사제들
error count : 25  겟 아웃
error count : 26  겨울왕국
error count : 26  겨울왕국
er

In [20]:
# 3)
data = {
    '영화명':movieName_List,
    '감독': director_List,
    '주연배우': actors_List,
    '평점': score_List,
    '평가자수':netizen_List,
    '상영시간':runtime_List,
    }
naverMovieInfo = pd.DataFrame(data)
naverMovieInfo.tail()

,영화명,감독,주연배우,평점,평가자수,상영시간
2392,히트,이성한,[마테우스 파체비치],8.12,246,20
2393,히트맨,최원섭,"[권상우, 정준호, 황우슬혜, 이이경, 이지원]",6.76,12618,110
2394,히트맨: 에이전트 47,알렉산더 바흐,"[재커리 퀸토, 루퍼트 프렌드, 시아란 힌즈, 한나 웨어]",7.90,1163,98
2395,힐빌리의 노래,론 하워드,"[에이미 아담스, 글렌 클로즈, 가브리엘 바쏘, 헤일리 베넷]",8.86,210,116
2396,"힘을 내요, 미스터 리",이계벽,"[차승원, 엄채영, 박해준]",7.71,6124,111


In [24]:
naverMovieInfo.to_csv("./naver_csv.csv",encoding="utf-8-sig")

In [25]:
'''여기서부터는 nono'''
# 수집할 데이터 : 주연배우, 평점, 평가자수, 상영시간
director_info = soup.find_all(class_='etc')[1].get_text()
director = director_info.split('|')[0][5:]
director
movie_info_s = soup.find("ul",class_='search_list_1').find_all(class_='etc')[1].get_text().split('|')
# 감독
director = movie_info_s[0][5:]
director
#영화배우
actors = movie_info_s[1][5:].split(',')
actors
score_info = soup.find("ul",class_='search_list_1').find('dd',class_='point')
# 평점
score = score_info.get_text()[1:5]
# 평가자수
netizen = score_info.get_text().split(' ')[-1][:-3]
netizen

'6124'

In [ ]:
html = "https://movie.naver.com/movie/search/result.naver?section=movie&query={search_movie}"
req = Request(html.format(search_movie=urllib.parse.quote("I AM"))) 
response = urlopen(req)
soup = BeautifulSoup(response,"html.parser")
#print(response, response.status)
for i in range(0,len(soup.find_all(class_='etc')),2): #n개 영화면 2n까지
    tmp_director = soup.find_all(class_='etc')[i+1].get_text().split('|')[0][5:]
    if tmp_director == real_director: #감독일치하면
        print("if func: ",tmp_director,real_director)
        #try:
        movie_info_s = soup.find("ul",class_='search_list_1').find_all(class_='etc')[i+1].get_text().split('|')
        # 주연배우
        print(movie_info_s[0])
        actors = movie_info_s[1][5:].split(',')
        # 평점
        score_info = soup.find("ul",class_='search_list_1').find('dd',class_='point')
        score = score_info.get_text()[1:5]
        # 평가자 수
        netizen = score_info.get_text().split(' ')[-1][:-3]
        #상영시간
        runtime = soup.find("ul",class_='search_list_1').find_all(class_='etc')[0].get_text().split('|')[2][:-2]
        movieName_List.append(movieName)
        director_List.append(tmp_director)
        actors_List.append(actors)
        score_List.append(score)
        netizen_List.append(netizen)
        runtime_List.append(runtime)
        break
        # except:
        #     print("inner error")
        #     print(len(soup.find_all(class_='etc')))
        #     print(soup.find_all(class_='etc')[i+1].get_text().split('|')[0][5:])
        #     continue
    else :
        print(i,"error",movieName)
        print(real_director,":",tmp_director,real_director==tmp_director)
        print("--")